<a href="https://colab.research.google.com/github/YASHxK1/code-a-thon-project/blob/main/this%20file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

using this

In [2]:
    # Import necessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Create a sample dataset (you can replace this with your own dataset)
def create_sample_dataset():
    conversations = [
        ("I've been feeling really down lately", "I understand you're going through a difficult time. Can you tell me more about what's been happening?"),
        ("I can't sleep at night", "Sleep difficulties can be really challenging. Have you noticed any patterns in your sleep disturbance?"),
        ("I'm constantly worried about everything", "It sounds like you're experiencing anxiety. Let's explore what specific situations trigger these worries."),
        ("I feel overwhelmed with work", "Managing work stress can be difficult. What aspects of your work are causing the most pressure?"),
        ("I keep having panic attacks", "That must be very distressing. Can you describe what happens during these panic attacks?"),
        ("I feel lonely all the time", "Feeling lonely can be very painful. What kind of social connections do you currently have?"),
        ("I'm having relationship problems", "Relationship challenges can be complex. Could you share more about what's happening?"),
        ("I feel worthless", "I hear that you're struggling with self-worth. What experiences have contributed to these feelings?"),
        ("I can't focus on anything", "Concentration difficulties can be frustrating. When did you first notice these changes?"),
        ("I'm grieving the loss of a loved one", "I'm so sorry for your loss. Would you like to tell me about your loved one?")
    ]
    # Multiply the dataset to create more training examples
    expanded_conversations = conversations * 50  # This will create 500 conversation pairs
    return expanded_conversations


In [4]:
# Text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s\?]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Data preparation class
class DataPreparator:
    def __init__(self, max_sequence_length=50):
        self.tokenizer = None
        self.max_sequence_length = max_sequence_length

    def prepare_data(self, conversations):
        # Split conversations into questions and answers
        questions, answers = zip(*conversations)

        # Preprocess texts
        questions = [preprocess_text(q) for q in questions]
        answers = [preprocess_text(a) for a in answers]

        # Add start and end tokens to answers
        answers = [f'<start> {a} <end>' for a in answers]

        # Create and fit tokenizer
        self.tokenizer = Tokenizer(filters='')
        self.tokenizer.fit_on_texts(questions + answers)

        # Convert texts to sequences
        encoder_input_data = self.tokenizer.texts_to_sequences(questions)
        decoder_input_data = self.tokenizer.texts_to_sequences(answers)

        # Pad sequences
        encoder_input_data = pad_sequences(encoder_input_data, maxlen=self.max_sequence_length, padding='post')
        decoder_input_data = pad_sequences(decoder_input_data, maxlen=self.max_sequence_length, padding='post')

        # Create decoder output data (shifted by one position)
        decoder_output_data = np.zeros_like(decoder_input_data)
        decoder_output_data[:, :-1] = decoder_input_data[:, 1:]

        return encoder_input_data, decoder_input_data, decoder_output_data

    @property
    def vocab_size(self):
        return len(self.tokenizer.word_index) + 1



In [5]:
# Mental Health Consultation Model
class MentalHealthLLM:
    def __init__(self, vocab_size, max_sequence_length, embedding_dim=256, lstm_units=256):
        self.vocab_size = vocab_size
        self.max_sequence_length = max_sequence_length
        self.embedding_dim = embedding_dim
        self.lstm_units = lstm_units
        self.encoder_model = None
        self.decoder_model = None
        self.model = self.build_model()

    def build_model(self):
        # Encoder
        encoder_inputs = Input(shape=(self.max_sequence_length,))
        encoder_embedding = Embedding(self.vocab_size, self.embedding_dim)(encoder_inputs)
        encoder_lstm = LSTM(self.lstm_units, return_state=True, return_sequences=True)
        encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
        encoder_states = [state_h, state_c]

        # Add self-attention to encoder
        attention = MultiHeadAttention(num_heads=8, key_dim=32)
        attention_output = attention(encoder_outputs, encoder_outputs)
        encoder_outputs = LayerNormalization()(attention_output + encoder_outputs)

        # Decoder
        decoder_inputs = Input(shape=(self.max_sequence_length,))
        decoder_embedding = Embedding(self.vocab_size, self.embedding_dim)
        decoder_lstm = LSTM(self.lstm_units, return_sequences=True, return_state=True)

        # Connect decoder to encoder
        decoder_embedded = decoder_embedding(decoder_inputs)
        decoder_outputs, _, _ = decoder_lstm(decoder_embedded, initial_state=encoder_states)

        # Add attention between encoder and decoder
        attention = MultiHeadAttention(num_heads=8, key_dim=32)
        attention_output = attention(decoder_outputs, encoder_outputs)
        decoder_outputs = LayerNormalization()(attention_output + decoder_outputs)

        # Add dropout for regularization
        decoder_outputs = Dropout(0.5)(decoder_outputs)
        decoder_dense = Dense(self.vocab_size, activation='softmax')
        decoder_outputs = decoder_dense(decoder_outputs)

        # Create the full model
        model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

        # Compile the model
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )



In [7]:
# Mental Health Consultation Model
class MentalHealthLLM:
    def __init__(self, vocab_size, max_sequence_length, embedding_dim=256, lstm_units=256):
        self.vocab_size = vocab_size
        self.max_sequence_length = max_sequence_length
        self.embedding_dim = embedding_dim
        self.lstm_units = lstm_units
        self.encoder_model = None
        self.decoder_model = None
        self.model = self.build_model()

    def build_model(self):
        # Encoder
        encoder_inputs = Input(shape=(self.max_sequence_length,))
        encoder_embedding = Embedding(self.vocab_size, self.embedding_dim)(encoder_inputs)
        encoder_lstm = LSTM(self.lstm_units, return_state=True, return_sequences=True)
        encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
        encoder_states = [state_h, state_c]

        # Add self-attention to encoder
        attention = MultiHeadAttention(num_heads=8, key_dim=32)
        attention_output = attention(encoder_outputs, encoder_outputs)
        encoder_outputs = LayerNormalization()(attention_output + encoder_outputs)

        # Decoder
        decoder_inputs = Input(shape=(self.max_sequence_length,))
        decoder_embedding = Embedding(self.vocab_size, self.embedding_dim)
        decoder_lstm = LSTM(self.lstm_units, return_sequences=True, return_state=True)

        # Connect decoder to encoder
        decoder_embedded = decoder_embedding(decoder_inputs)
        decoder_outputs, _, _ = decoder_lstm(decoder_embedded, initial_state=encoder_states)

        # Add attention between encoder and decoder
        attention = MultiHeadAttention(num_heads=8, key_dim=32)
        attention_output = attention(decoder_outputs, encoder_outputs)
        decoder_outputs = LayerNormalization()(attention_output + decoder_outputs)

        # Add dropout for regularization
        decoder_outputs = Dropout(0.5)(decoder_outputs)
        decoder_dense = Dense(self.vocab_size, activation='softmax')
        decoder_outputs = decoder_dense(decoder_outputs)

        # Create the full model
        model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

        # Compile the model
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        # Create inference models
        self.encoder_model = Model(encoder_inputs, [encoder_outputs] + encoder_states)

        decoder_state_input_h = Input(shape=(self.lstm_units,))
        decoder_state_input_c = Input(shape=(self.lstm_units,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

        decoder_outputs, state_h, state_c = decoder_lstm(
            decoder_embedded,
            initial_state=decoder_states_inputs
        )
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states
        )

        return model

    def train(self, encoder_input_data, decoder_input_data, decoder_output_data,
              batch_size=32, epochs=100, validation_split=0.2): # Corrected indentation

        # Implement early stopping and model checkpointing
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True
            ),
            ModelCheckpoint(
                'best_model.h5',
                monitor='val_loss',
                save_best_only=True
            )
        ]

        # Train the model
        history = self.model.fit(
            [encoder_input_data, decoder_input_data],
            decoder_output_data,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=validation_split,
            callbacks=callbacks
        )

        return history

In [ ]:


# Main execution
def main():
    print("Creating dataset...")
    conversations = create_sample_dataset()

    print("Preparing data...")
    data_preparator = DataPreparator(max_sequence_length=50)
    encoder_input_data, decoder_input_data, decoder_output_data = data_preparator.prepare_data(conversations)

    print("Building and training model...")
    model = MentalHealthLLM(
        vocab_size=data_preparator.vocab_size,
        max_sequence_length=50,
        embedding_dim=256,
        lstm_units=256
    )

    history = model.train(
        encoder_input_data,
        decoder_input_data,
        decoder_output_data,
        batch_size=32,
        epochs=50,
        validation_split=0.2
    )

    print("\nTesting the model with sample inputs...")
    test_inputs = [
        "I've been feeling really sad lately",
        "I'm having trouble sleeping",
        "I feel anxious all the time"
    ]

    for test_input in test_inputs:
        response = model.generate_response(test_input, data_preparator.tokenizer)
        print(f"\nInput: {test_input}")
        print(f"Response: {response}")

if __name__ == "__main__":
    print("Starting Mental Health LLM training...")
    main()


Starting Mental Health LLM training...
Creating dataset...
Preparing data...
Building and training model...
Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 746ms/step - accuracy: 0.5119 - loss: 3.1147

13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 876ms/step - accuracy: 0.5200 - loss: 3.0547 - val_accuracy: 0.6940 - val_loss: 1.4190
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 625ms/step - accuracy: 0.6946 - loss: 1.5547

13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 692ms/step - accuracy: 0.6952 - loss: 1.5497 - val_accuracy: 0.7420 - val_loss: 1.1366
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 617ms/step - accuracy: 0.7241 - loss: 1.2469

13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 683ms/step - accuracy: 0.7245 - loss: 1.2424 - val_accuracy: 0.7700 - val_loss: 0.8232
Epoch 4/50
 2/13 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.7541 - loss: 0.9955